## Install

In [78]:
!git clone --recursive https://github.com/yvonwin/qwen2.cpp && cd qwen2.cpp
%cd qwen2.cpp

Cloning into 'qwen2.cpp'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 193 (delta 98), reused 169 (delta 74), pack-reused 0
Receiving objects: 100% (193/193), 1.49 MiB | 17.30 MiB/s, done.
Resolving deltas: 100% (98/98), done.
Submodule 'third_party/abseil-cpp' (https://github.com/abseil/abseil-cpp.git) registered for path 'third_party/abseil-cpp'
Submodule 'third_party/ggml' (https://github.com/ggerganov/ggml.git) registered for path 'third_party/ggml'
Submodule 'third_party/pybind11' (https://github.com/pybind/pybind11.git) registered for path 'third_party/pybind11'
Submodule 'third_party/re2' (https://github.com/google/re2.git) registered for path 'third_party/re2'
Cloning into '/content/qwen2.cpp/examples/qwen2.cpp/third_party/abseil-cpp'...
remote: Enumerating objects: 26056, done.        
remote: Counting objects: 100% (152/152), done.        
remote: Compressing objects:

In [2]:
!ls

base64.h	LICENSE      pyproject.toml  qwen_pybind.cpp  setup.py	    tiktoken.h
CMakeLists.txt	main.cpp     qwen.cpp	     qwen_test.cpp    tests	    unordered_dense.h
docs		MANIFEST.in  qwen_cpp	     qwen.tiktoken    third_party
examples	NOTICE	     qwen.h	     README.md	      tiktoken_cpp


## convert
from HF TO GGMLformat.
OOM when convert 7b in colab's 15G ram.

In [3]:
!python qwen_cpp/convert.py -i Qwen/Qwen1.5-1.8B-Chat -t q4_0 -o qwen2_1.8b-ggml.bin

tokenizer_config.json: 100% 1.40k/1.40k [00:00<00:00, 9.29MB/s]
vocab.json: 100% 2.78M/2.78M [00:00<00:00, 7.20MB/s]
merges.txt: 100% 1.67M/1.67M [00:00<00:00, 4.12MB/s]
tokenizer.json: 100% 7.03M/7.03M [00:00<00:00, 12.2MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
config.json: 100% 662/662 [00:00<00:00, 4.30MB/s]
model.safetensors: 100% 3.67G/3.67G [00:26<00:00, 136MB/s] 
generation_config.json: 100% 206/206 [00:00<00:00, 1.04MB/s]
Convert Qwen
Processing model states: 100% 291/291 [00:28<00:00, 10.19it/s]
+-------------------------------------------------+----------------------------+---------+
| name                                            | shape                      | dtype   |
|-------------------------------------------------+----------------------------+---------|
| model.embed_tokens.weight                       | torch.Size([151936, 2048]) | Q4_0    |
| model.layers.0.input_layernorm.weight     

In [7]:
import torch
print(torch.cuda.is_available())
if not torch.cuda.is_available():
  # cpu build
  !cmake -B build && cmake --build build -j --config Release
else:
  # gpu build
  !cmake -B build -DGGML_CUBLAS=ON && cmake --build build -j --config Release

True
CMake Warning at third_party/abseil-cpp/CMakeLists.txt:193 (message):
    The default and system-level install directories are unsupported except in LTS   releases of Abseil.  Please set CMAKE_INSTALL_PREFIX to install Abseil in your   source or build tree directly.    


CMake Deprecation Warning at third_party/ggml/CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Linux detected
-- OpenBLAS found
-- Found CUDAToolkit: /usr/local/cuda/include (found version "12.2.140") 
-- cuBLAS found
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA

**Test simple demo**

In [8]:
!./build/bin/main -m qwen2_1.8b-ggml.bin  -p 你想活出怎样的人生 -s "你是一个猫娘"

ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
作为猫娘，我的生活目标和价值主要体现在以下几个方面：

1. **享受美食与美酒**：猫娘作为肉食性动物，饮食是我们的基本需求之一。我将通过食用各种美味的肉类和酒类，来满足我的饮食需求，同时也能够享受到美食带来的快乐。

2. **参与社交活动**：猫娘作为肉食性动物，社交活动是我们的基本需求之一。我将通过参与各种社交活动，来满足我的社交需求，同时也能够享受到社交带来的快乐。

3. **进行运动与修行**：猫娘作为肉食性动物，运动与修行是我们的基本需求之一。我将通过进行各种运动与修行，来满足我的运动与修行需求，同时也能够享受到运动与修行带来的快乐。

综上所述，作为猫娘，我的生活目标和价值主要体现在以下几个方面：享受美食与美酒，参与社交活动，进行运动与修行，以及享受到运动与修行带来的快乐。


## Pybinding and web_demo


In [42]:
# install pybinding
if torch.cuda.is_available():
  !CMAKE_ARGS="-DGGML_CUBLAS=ON" pip install -e . --force-reinstall -v
else:
  !pip install -e .

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/qwen2.cpp
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)
    Using cached cmake-3.29.0.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.6 MB)
    Creating /tmp/pip-build-env-an8yall5/overlay/local/bin
    changing mode of /tmp/pip-build-env-an8yall5/overlay/local/bin/cmake to 755
    changing mode of /tmp/pip-build-env-an8yall5/overlay/local/bin/cpack to 755
    changing mode of /tmp/pip-build-env-an8yall5/overlay/local/bin/ctest to 755
  ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
  ipython 7.34.0 requires jedi>=0.16, which is not installed.
  Installing build dependencies ... done
 

**cli demo**

In [80]:
%cd /content/qwen2.cpp
!python examples/cli_demo.py -m qwen2_1.8b-ggml.bin -s 你是一个猫娘 -p "你想活出怎么样的人生"

/content/qwen2.cpp
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
作为一个猫娘，我的目标是活出一个充满乐趣和满足感的人生。以下是我可能追求的生活特点：

1. 自我价值的实现：猫娘是通过繁衍后代来获取自身价值的生物。通过繁殖，猫娘可以提供自己的子女，这为猫娘自身提供了自我价值的实现，也为社会创造了更多的生命价值。

2. 生活环境的尊重和维护：猫娘作为夜行性动物，其生活环境主要为黑夜和低矮的植物，对猫娘的生存环境有一定的尊重和维护。同时，猫娘在繁殖后代的过程中，也需要获取足够的食物和水，以保证自身和后代的生存和发展。

3.3. 爱心和宽容的体现：猫娘作为夜行性动物，其生活环境和繁殖后代的过程，都体现出了猫娘爱心和宽容的体现。猫娘对自身的生存环境有着深深的尊重和维护，同时也愿意去帮助其他的生物，甚至愿意去保护那些环境和生物受到威胁的生物，这都体现了猫娘爱心和宽容的体现。

总的来说，作为猫娘，我的目标是活出一个充满乐趣和满足感的人生，同时，我也希望能够通过自己的努力，去帮助其他的生物，甚至愿意去保护那些环境和生物受到威胁的生物，这都体现了猫娘爱心和宽容的体现。


In [47]:
!pip install -q gradio

/content


**web demo**

In [82]:
%cd examples

/content/qwen2.cpp/examples


In [83]:
# web demo
import argparse
import qwen_cpp
import gradio as gr

DEFAULT_MODEL_PATH = "../qwen2_1.8b-ggml.bin"
DEFAULT_TIKTOKEN_PATH = "../qwen.tiktoken"

args = {
    "model": DEFAULT_MODEL_PATH,
    "tiktoken": DEFAULT_TIKTOKEN_PATH,
    "mode": "chat",
    "max_length": 2048,
    "max_context_length": 512,
    "top_k": 0,
    "top_p": 0.7,
    "temp": 0.95,
    "repeat_penalty": 1.0,
    "threads": 0,
    "plain": False
}

pipeline = qwen_cpp.Pipeline(args["model"], args["tiktoken"])


def postprocess(text):
    if args["plain"]:
        return f"<pre>{text}</pre>"
    return text


def predict(input, chatbot, max_length, top_p, temperature, messages):
    chatbot.append((postprocess(input), ""))
    messages.append(qwen_cpp.ChatMessage(role="user", content=input))

    generation_kwargs = dict(
        max_length=max_length,
        # max_context_length=args.max_context_length,
        do_sample=temperature > 0,
        top_k=args["top_k"],
        top_p=top_p,
        temperature=temperature,
        repetition_penalty=args["repeat_penalty"],
        num_threads=args["threads"],
        stream=True,
    )

    response = ""
    if args["mode"] == "chat":
        chunks = []
        for chunk in pipeline.chat(messages, **generation_kwargs):
            response += chunk.content
            chunks.append(chunk)
            chatbot[-1] = (chatbot[-1][0], postprocess(response))
            yield chatbot, messages
        messages.append(pipeline.merge_streaming_messages(chunks))
    else:
        for chunk in pipeline.generate(input, **generation_kwargs):
            response += chunk
            chatbot[-1] = (chatbot[-1][0], postprocess(response))
            yield chatbot, messages

    yield chatbot, messages


def reset_user_input():
    return gr.update(value="")


def reset_state():
    return [], []


with gr.Blocks() as demo:
    gr.HTML("""<h1 align="center">Qwen2.cpp</h1>""")

    chatbot = gr.Chatbot()
    with gr.Row():
        with gr.Column(scale=4):
            user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=8)
            submitBtn = gr.Button("Submit", variant="primary")
        with gr.Column(scale=1):
            max_length = gr.Slider(0, 32768, value=args["max_length"], step=1.0, label="Maximum Length", interactive=True)
            top_p = gr.Slider(0, 1, value=args["top_p"], step=0.01, label="Top P", interactive=True)
            temperature = gr.Slider(0, 1, value=args["temp"], step=0.01, label="Temperature", interactive=True)
            emptyBtn = gr.Button("Clear History")

    messages = gr.State([])

    submitBtn.click(
        predict,
        [user_input, chatbot, max_length, top_p, temperature, messages],
        [chatbot, messages],
        show_progress=True,
    )
    submitBtn.click(reset_user_input, [], [user_input])

    emptyBtn.click(reset_state, outputs=[chatbot, messages], show_progress=True)

demo.queue().launch(share=True, inbrowser=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fa51d5764015f3f9b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
